In [185]:
import pickle
import os

def loadData(pickle_name):
    # for reading also binary mode is important
    dbfile = open(pickle_name, 'rb')    
    db = pickle.load(dbfile)
    dbfile.close()
    return db
    
# print(loadData("../re.pickle"))

db = loadData("../re.pickle")

def create_sum(data):
    grouped_dict = {}
    for x in db:
        if x['batch_size'] not in grouped_dict:
            grouped_dict[x['batch_size']] = [x]
        else:
            grouped_dict[x['batch_size']].append(x)
    
    return grouped_dict

with pd.ExcelWriter('output-1.xlsx') as writer:
    cols = ["Avg", "1st", "2nd"] 
    bs_class = create_sum(db)
    # print(bs_class)
    for key, value in bs_class.items():
        # print(value)
        
        for bs_sample in value:
            sheet_name = '{0}_{1}'.format(bs_sample['precision'], int(key))
            sun = list(bs_sample['kpi'].keys())
        index = [i for i in sun]
        dist = [ pd.DataFrame(list(bs_value['kpi'].values()) , columns=cols, index=index) for bs_value in value ]
        x= pd.concat(dist, keys=['50', '52', '54', '56'])
        x.to_excel(writer, sheet_name=sheet_name, index_label=['core', 'fre'])
        # print(x.to_excel(writer, sheet_name=sheet_name, index_label="Fre"))
        #df = pd.DataFrame(result_df, columns=cols, index=index)
        # df.concat(dist)
        # pd.concat(dist).to_excel(writer)
        #df.to_excel(writer, sheet_name=sheet_name, index_label="Fre")

        
            
        
        
        
    # for value in db:
    #     result_df = list(value['kpi'].values())
    #     sun = list(value['kpi'].keys())
    #     index = [i for i in sun]
    #     sheet_name = '{0}_{1}_{2}'.format(value['core'], value['precision'], value['batch_size'])
        
    #     sheet_list.append(sheet_name)       
    #     df = pd.DataFrame(result_df, columns=cols, index=index)
    #     df.to_excel(writer, sheet_name=sheet_name, index_label="Fre")


# for key, value in my_dict.items():
#     value = value.split('_')[-1]
#     if value not in grouped_dict:
#         grouped_dict[value] = [key]
#     else:
#         grouped_dict[value].append(key)

In [71]:
import re

def parse_to_excel(platform, report_path, log_file_name):
    """
    parse log to excel
    """
    logfile_result = parse_log_name(log_file_name)
    logfile_fullpath = os.path.join(report_path,log_file_name)
    single_file_dict = {}
    #Read output log file
    with open(logfile_fullpath, 'r') as ds_log:
        lines = ds_log.readlines()
        for line in lines:
            # Get latency
            if re.search("Inference latency:", line):
                single_loop_list = []
                latency=re.findall('\d+\.\d+', line)[0]
                # single_loop_list.append(latency)
                #print("latency:", float(latency))
            # Get first_token
            if re.search("First token average latency:", line):
                first_token=re.findall('\d+\.\d+', line)[0]
                single_loop_list.append(first_token)
                #print("first_token:", float(first_token))
            if re.search("Average 2... latency:", line):
                second_token=re.findall('\d+\.\d+', line)[0]
                single_loop_list.append(second_token)
                #print("second_token:",float(second_token))
            if re.search("WSF Portal URL:", line):
                dashboard_link = re.findall('https://.*', line)[0]
                # single_loop_list.append({dashboard_link})
                #print(dashboard_link)
            if re.search("Current\s+\d+-\d+-\w+.*", line):
                loop_time=re.findall('([0-9])', line)[0]
                current_frequency=re.findall('\d.\dGhz', line)[0]
                #print("loop_time and current_frequency:", loop_time, current_frequency)
                link = '=HYPERLINK("{0}", "{1}")'.format(dashboard_link, latency)
                single_loop_list.insert(0, link)
                single_file_dict[current_frequency] = single_loop_list
    
    logfile_result['kpi'] = single_file_dict
    return logfile_result

value = parse_to_excel("spr", "../report/spr/", "c_50_bfloat16_1.log")

def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,name)


with pd.ExcelWriter('output.xlsx') as writer:
    # for value in last_re:
    result_df = list(value['kpi'].values())
    #result_df = [r['kpi_values'] for r in list(value['kpi'].values())]
    print(result_df)
    sun = list(value['kpi'].keys())
    index = [i for i in sun]
    print(index)
    sheet_name = '{0}_{1}_{2}'.format(value['core'], value['precision'], value['batch_size'])
    cols = ["Avg", "1st", "2nd"]                  
    df = pd.DataFrame(result_df, columns=cols, index=index)
    # df['Avg'] = df.apply(lambda x: make_clickable(x['url'], x['name']), axis=1)
    df.to_excel(writer, sheet_name=sheet_name, index_label="Fre")


[['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/40a988aa-1b1b-48aa-b4d1-a9dc8a7cd741", "8.648")', '0.890', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/69bd49e3-78bc-4bf8-80a3-1bb43c0597c1", "8.641")', '0.891', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/6d1cc3b0-b366-49de-b44e-f13595927d3f", "8.626")', '0.901', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/c38a2da4-783a-483c-8d4d-12039d375d2f", "8.644")', '0.894', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/aeb864e4-3a35-4575-8d78-6cc1a0d735cb", "8.649")', '0.897', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/ebfa5e4c-a338-4f38-a0f5-63bd90e275da", "8.650")', '0.897', '0.061']]
['2.8Ghz', '3.0Ghz', '3.2Ghz', '3.4Ghz', '3.6Ghz', '3.8Ghz']


In [72]:
import pandas as pd
df = pd.DataFrame([['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/40a988aa-1b1b-48aa-b4d1-a9dc8a7cd741", "8.648")', '0.890', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/69bd49e3-78bc-4bf8-80a3-1bb43c0597c1", "8.641")', '0.891', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/6d1cc3b0-b366-49de-b44e-f13595927d3f", "8.626")', '0.901', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/c38a2da4-783a-483c-8d4d-12039d375d2f", "8.644")', '0.894', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/aeb864e4-3a35-4575-8d78-6cc1a0d735cb", "8.649")', '0.897', '0.061'], ['=HYPERLINK("https://wsf-dashboards.intel.com/services-framework/perfkitruns/run_uri/ebfa5e4c-a338-4f38-a0f5-63bd90e275da", "8.650")', '0.897', '0.061']])
df.to_excel('test.xlsx')

In [155]:
# Importing pandas package
import pandas as pd

# Creating a Dictionary
d = {
    'Product':['TV','Fridge','AC'],
    'Electronic':[True,False,False],
    'Eletric':[False,True,True]
}

# Creating DataFrame
df = pd.DataFrame(d)

# Display the DataFrame
print("Original DataFrame:\n",df,"\n")

# Creating an empty series
s = pd.Series([None,None,None],index=['Product','Electronic','Electric'])

# Appending empty series to df
result = df.append(s,ignore_index=True)

# Display modified DataFrame
print("Modified DataFrame:\n",result)

Original DataFrame:
   Product  Electronic  Eletric
0      TV        True    False
1  Fridge       False     True
2      AC       False     True 



AttributeError: 'DataFrame' object has no attribute 'append'